In [175]:
import requests
import geopandas as gpd
from shapely.geometry import Polygon



name = "hereapi"
ID = 'AeRgZacJKwd2aWmDMyhr'
CODE = '7PLnT1O28NidyurIk5BqJA'

head = 'https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json?'
URL_Base = '{}app_id={}&app_code={}&mode=shortest;car;traffic:{}&start=geo!{},{}&range={}&rangetype={}'
URL_Base_dep = '{}app_id={}&app_code={}&mode=shortest;car;traffic:{}&departure={}&start=geo!{},{}&range={}&rangetype={}'

def get_isodata(location, range_iso = 3600, type_iso = 'time', traffic='disabled', departure=None): #receives a single location
    if departure == None:
        url = URL_Base.format(head, ID, CODE, traffic, location[0], location[1], range_iso, type_iso) 
    else: 
        url = URL_Base_dep.format(head, ID, CODE, traffic, departure, location[0], location[1], range_iso, type_iso)
    try: 
        js = requests.get(url).json()['response']
        iso = js['isoline']
        coords = Polygon([(float(x.split(',')[1]), float(x.split(',')[0])) for x in iso[0]['component'][0]['shape']])
        geojs = gpd.GeoSeries([coords]).__geo_interface__
        geojs['style'] = {"__comment": "all SVG styles allowed", "fill":"red", "stroke-width":"3", "fill-opacity":0.6
    }
        return geojs
    except KeyError:
        js = requests.get(url).json()
        print(js)
        raise ValueError("HereAPI doesn't have data requested")

def isoplot(center, locations, color, outputfile):
    fm = folium.Map(location = center, zoom_start=10, tiles='CartoDBPositron')
    folium.GeoJson(locations).add_to(fm)
    fm.save(outputfile)
    print('map saved to %s' %outputfile)
    
    
def get_isodata_list(locations, range_iso = 3600, type_iso = 'time', traffic = 'disabled'): 
    #recevies many locations, as a list of tuples of locations
    isoclines = []
    for loc in locations: 
        try:
            line = get_isodata(loc, range_iso, type_iso, traffic)
            isoclines.append(line)
        except ValueError:
            pass
    return isoclines

def isoplots(centers, locations, names, color):
    for i, center in enumerate, centers:
        isoplot(center, locations[i], color, names[i]+'.html')
        
def make_list(centers, names, color, range_iso = 3600, type_iso = 'time', traffic = 'disabled'):
    for i, center in enumerate(centers):
        try:
            line = get_isodata(center, range_iso, type_iso, traffic)    
        except ValueError:
            pass
        isoplot(center, line, 'red', 'isoclines/' + names[i] +'.html')
        
        
    

In [213]:
import folium
import pytz
import datetime

ist = pytz.timezone('Asia/Kolkata')

rush_hour_ist = datetime.datetime(2019,2,13,17,0,0,tzinfo=ist).isoformat()
print(rush_hour_ist)
abad_center = (23.027496, 72.572697)
locsabad = get_isodata(abad_center, 3600, 'time')

isoplot(abad_center, locsabad,'red','out.html')

mumbai_center = (19.060828, 72.872478)
locs = get_isodata(mumbai_center, traffic='enabled', departure='2019-02-13T17:00:00')
isoplot(mumbai_center, locs, 'red', 'mumbai_traffic.html')
locs2 = get_isodata(mumbai_center, traffic='disabled', departure = '2019-02-13T23:00:00')
isoplot(mumbai_center, locs2, 'red', 'mumbai_notraffic2.html')


#hereapi.isoplot(home, locs,'red','out.html')

2019-02-13T17:00:00+05:53
map saved to out.html
map saved to mumbai_traffic.html
map saved to mumbai_notraffic2.html


In [212]:
import csv 

city_centers = []
names = []
with open('worldcities.csv') as worldcities:
    csv_reader = csv.reader(worldcities, delimiter=',')
    for i, row in enumerate(csv_reader):
        if i==0:
            pass
        else:
            if row[9] == '' or float(row[9]) < 1000000:
                pass
            else:
                city_centers.append((row[2], row[3]))
                names.append(row[0])


In [207]:
from functools import partial
import pyproj
from shapely.geometry import shape, GeometryCollection
from shapely.ops import transform
import json

#not sure about how well this function works. 

def geojson_area(geojs):
    js = geojs['features'][0]['geometry']
    s = shape(js)
    proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
    s_new = transform(proj, s)
    return transform(proj, s).area



In [214]:
geojson_area(locsabad)

2106390185.5910017